In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
sns.set_style('ticks')
sns.set_context('paper')

In [ ]:
res = pd.read_csv('../outputs/bluewaters.csv')

In [ ]:
res['nproc'] = res.matsize / res.chunksize

In [ ]:
res

In [ ]:
for i in res.groupby('matsize'):
    print(i)

In [ ]:
from itertools import cycle

In [ ]:
sns.palplot(sns.color_palette('Set1', 9, desat=0.6))

In [ ]:
shapes = cycle(('o', 's', '^', 'v', '<', 'D'))

with sns.color_palette('Paired'):
    for i, (m, df) in enumerate(res.groupby('matsize')):
        plt.plot(df.nproc, df.agtime, marker=next(shapes), label='M={}, AG'.format(m))
        plt.plot(df.nproc, df.ringtime, marker=next(shapes), linestyle='--', label='M={}, Ring'.format(m))

    plt.semilogy()
    
    sns.despine()
#     plt.grid(which='both')
    
    plt.legend(loc=8, ncol=3)

In [ ]:
shapes = cycle(('o', 's', '^', 'v', '<', 'D'))

plt.figure(figsize=(345/72, 345/72*0.7))

with sns.color_palette('Paired', 8):
    for i, (m, df) in enumerate(res.groupby('nproc')):
        plt.plot(df.matsize, df.agtime, marker=next(shapes), label='p={:.0f}, AG'.format(m))
        plt.plot(df.matsize, df.ringtime, marker=next(shapes), linestyle='--', label='p={:.0f}, Ring'.format(m))

    plt.loglog(basex=2)
    
    sns.despine()
#     plt.grid(which='both')
    
    plt.legend(loc=2, ncol=2)
    plt.xlabel('Matrix size')
    plt.ylabel('Time [s]')
    
    plt.tight_layout()
    plt.savefig('data.pdf')

In [ ]:
plt.figure(figsize=(345/72, 345/72*0.7))

with sns.color_palette('Paired', 9):
    xs = np.logspace(10, 17, base=2)
    for p in res.nproc.unique():
        plt.plot(xs, ag_model(xs, p), label='p={:.0f}, AG'.format(p))
        plt.plot(xs, [ring_model(m, p) for m in xs], '--', label='p={:.0f}, Ring'.format(p))

    plt.loglog(basex=2)
    
    sns.despine()
    plt.legend(loc=2, ncol=2)
    
    plt.xlabel('Matrix size')
    plt.ylabel('Time [s]')
    
    plt.tight_layout()
    plt.savefig('model.pdf')

In [ ]:
with sns.color_palette('Paired', 9):
    xs = np.logspace(7, 10, base=2)
    for m in res.matsize.unique():
        plt.plot(xs, ag_model(m, xs))
        plt.plot(xs, [ring_model(m, p) for p in xs], '--')

    plt.loglog(basex=2)
#     plt.semilogy()

In [ ]:
def compute_time(m, p, r=7.8e-11, c=5.1e-11):
    return 2*m**2 / p * (r+c) + 2 * m * r

def net_time(m, p, r=6.8e-10):
    return r * 8 * m / p

def ag_model(m, p):
    return net_time(m, p) * p + compute_time(m, p)

def ring_model(m, p):
    return max(p * net_time(m, p), compute_time(m, p))

In [ ]:
compute_time(102400, 512)

In [ ]:
net_time(102400, 512)

In [ ]:
xs = np.